In [1]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

<Logger detectron2 (DEBUG)>

In [2]:
import detectron2
import numpy as np
import os, json, cv2, random
# from google.colab.patches import cv2_imshow
from IPython import display
import PIL


# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [1]:
#!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="rpYN39AN5etZ9aS7Zzi5")
project = rf.workspace("roboflow-100").project("road-signs-6ih4y")
dataset = project.version(1).download("coco")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to road-signs-1 in coco:: 100%|██████████| 2101/2101 [00:00<00:00, 2648.42it/s]


As my dataset was in 0-based category index, I had to increase the category indices by 1, using the following code block:

In [10]:
%pwd

'd:\\DATASET\\detectron2'

In [13]:
from detectron2.data import DatasetCatalog, MetadataCatalog

# Remove existing datasets if already registered
for dataset_name in ["my_dataset_train", "my_dataset_val", "my_dataset_test"]:
    if dataset_name in DatasetCatalog:
        DatasetCatalog.remove(dataset_name)
    if dataset_name in MetadataCatalog:
        MetadataCatalog.remove(dataset_name)

# Register datasets
from detectron2.data.datasets import register_coco_instances

register_coco_instances("my_dataset_train", {}, r"D:\DATASET\detectron2\road-signs-1\train\annotations.coco.json", r"D:\DATASET\detectron2\road-signs-1\train")
register_coco_instances("my_dataset_val", {}, r"D:\DATASET\detectron2\road-signs-1\valid\annotations.coco.json", r"D:\DATASET\detectron2\road-signs-1\valid")
register_coco_instances("my_dataset_test", {}, r"D:\DATASET\detectron2\road-signs-1\test\annotations.coco.json", r"D:\DATASET\detectron2\road-signs-1\test")


AssertionError: Attribute 'json_file' in the metadata of 'my_dataset_train' cannot be set to a different value!
D:\DATASET\detectron2oad-signs-1	rain\_annotations.coco.json != /road-signs-1/train/annotations.coco.json

In [6]:
# import some common detectron2 utilities
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os, json, cv2, random
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/retinanet_R_50_FPN_1x.yaml"))
cfg.DATASETS.TRAIN = ("my_dataset_train",)
cfg.DATASETS.TEST = ("my_dataset_val",)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/retinanet_R_50_FPN_1x.yaml")  # Let training initialize from model zoo
cfg.SOLVER.IMS_PER_BATCH = 8  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.001  # pick a good LR
cfg.SOLVER.MAX_ITER = 5000    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.RETINANET.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.RETINANET.NUM_CLASSES = 22  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg)
trainer.resume_or_load(resume=False)
trainer.train()

Loading config d:\DATASET\detectron2\env\Lib\site-packages\detectron2\model_zoo\configs\COCO-Detection\../Base-RetinaNet.yaml with yaml.unsafe_load. Your machine may be at risk if the file contains malicious content.


[12/02 12:10:56 d2.engine.defaults]: Model:
RetinaNet(
  (backbone): FPN(
    (fpn_lateral3): Conv2d(512, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output3): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral4): Conv2d(1024, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output4): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (fpn_lateral5): Conv2d(2048, 256, kernel_size=(1, 1), stride=(1, 1))
    (fpn_output5): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (top_block): LastLevelP6P7(
      (p6): Conv2d(2048, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (p7): Conv2d(256, 256, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
    )
    (bottom_up): ResNet(
      (stem): BasicStem(
        (conv1): Conv2d(
          3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
          (norm): FrozenBatchNorm2d(num_features=64, eps=1e-05)
        )
      )
      (res2)

OSError: [Errno 22] Invalid argument: 'D:\\DATASET\\detectron2\road-signs-1\train\\_annotations.coco.json'